# ANN for House proce prediction - extended to save the pretrained model
This notebook extends the functionality of the original model by "saving final weights to later use as tflite" and "integrating with GCP Vertex AI for efficient training".
Original code [link text](https://github.com/Prajwal10031999/House-Price-Prediction-with-ANN)

In [ ]:
# Imports
import numpy as np
import pandas as pd


In [2]:
# Upload files to Vertex colab
from google.colab import files
uploaded = files.upload()

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
# NN imports
# same as original

In [ ]:

# Create NN model - same
model = Sequential()

model.add(Dense(units=256, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(units=128, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(units=64, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(units=32, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(units=16, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(units=8, activation='linear'))
model.add(BatchNormalization())
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mean_squared_logarithmic_error',
               optimizer='adam')

history_model = model.fit(X_train, Train_fey, validation_split=0.2, epochs=1000, batch_size=64, verbose=0)
predict = model.predict(X_test, batch_size=64)

pred = np.exp(predict).flatten()
pred = pd.DataFrame({'Id':Id_final, 'SalePrice':pred})
pred['SalePrice'] = pred['SalePrice'].transform(lambda x: (x-1))

pred.to_csv('submission.csv', index=False)


In [ ]:
# Access the trainng loss (error)
training_loss = history_model.history['loss']

# Access the validation loss (error)
validation_loss = history_model.history['val_loss']

# Print the training and validation errors
print("Training Loss:", training_loss)
print("Validation Loss:", validation_loss)

In [ ]:
# Graph for loss
import matplotlib.pyplot as plt
plt.plot(history_model.history['loss'])
plt.plot(history_model.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# print the model weights
for layer in model.layers:
    weights = layer.get_weights()
    if weights:    # Check if the layer has weights (some layers might not)
      print("Layer Name:", layer.name)
      print("Weights:")
      print(weights[0])    # Weight matrix
      print("Biases:")
      print(weights[1])    # Biases vector
      print("------------------")

In [ ]:
# save weights
model.save_weights('og_model.weights.h5')

In [ ]:
# the above mentioned returned an empty file. Here is what worked
import h5py
def save_weights_to_h5(model, output_file_name):
  """
  Saves weights and biases from a Keras model to an HDF5 file.

  Args:
      model: A Keras model.
      output_file_name: The name of the HDF5 file to save the weights to.

  Returns:
      None. """

  with h5py.File(output_file_name, 'w') as outfile:
    for i, layer in enumerate(model.layers):
      weights = layer.get_weights()
      if weights:
        layer_group = outfile.create_group(layer.name)
        layer_group.create_dataset('weights', data=weights[0])
        layer_group.create_dataset('bias', data=weights[1])

output_file = 'weights.h5'
save_weights_to_h5(model, output_file)

In [ ]:
# Open/Print the H5 file
def print_weights_from_h5(file_path):
  with h5py.File(file_path, 'r') as f:
    for layer_name in f.keys():
      layer_data = f(layer_name)
      print("Layer:", layer_name)
      print("  Weight:")
      print(np.array(layer_data['weight']))     # Print the weights
      print("  Bias:")
      print(np.array(layer_data['bias']))       # Print the biases
      print("------------------")

file_path = 'weights.h5'
print_weights_from_h5(file_path)

In [ ]:
# Download the file
from google.colab import files
files.download('weights.h5')